# The objective of this tutorial is to use the AppEEARS API to extract data to visualize clouds
### Adapted from: https://git.earthdata.nasa.gov/projects/LPDUR/repos/appeears-api-getting-started/browse

In [3]:
# Import packages 
import requests as r
import getpass, pprint, time, os, cgi, json # getpass needed for Earthdata login
import geopandas as gpd

In [4]:
# Set input directory, change working directory
inDir = '/home/marie/wildfire/'           # working directory
os.chdir(inDir)                                      # Change to working directory
api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable

## Login to AppEEARS

In [5]:
user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')      # Input NASA Earthdata Login Username
password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')  # Input NASA Earthdata Login Password

Enter NASA Earthdata Login Username: ········
Enter NASA Earthdata Login Password: ········


#### The requests package will post your username and password. The token received will be used later to submit your request.

In [6]:
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
del user, password                                                           # Remove user and password information
token_response                                                               # Print response

{'token_type': 'Bearer',
 'token': 'iygN52Rz8wtEEpwRUu8YEchCuw_mInrwxxKOLcf4PUEmdl-njOjvjuWbEnWdDPoYnYO9KLpU0NIfxsGtOvsuDg',
 'expiration': '2022-08-05T20:57:09Z'}

#### Available products from appears
https://appeears.earthdatacloud.nasa.gov/products

In [8]:
# The get() method returns the value of the item with the specified key.
product_response = r.get('{}product'.format(api)).json()                         # request all products in the product service
# print('AρρEEARS currently supports {} products.'.format(len(product_response)))  # Print no. products available in AppEEARS

#### I'm interested in LST because it has not been pre-processed for clouds, therefore, you can visualize them easier

In [9]:
# my_dict = {} # if you want to create an empty dictionary
# I think this is sort setting p to productandversion, then iterating through all productandversions
products = {p['ProductAndVersion']: p for p in product_response} # Create a dictionary indexed by product name & version
products['ECO2LSTE.001']                                         # Print information for the product of interest ECO2LSTE.001 LST Product
# print(products)

{'Product': 'ECO2LSTE',
 'Platform': 'ECOSTRESS',
 'Description': 'Land Surface Temperature & Emissivity (LST&E)',
 'RasterType': 'Swath',
 'Resolution': '70m',
 'TemporalGranularity': 'ISS-dependent',
 'Version': '001',
 'Available': True,
 'DocLink': 'https://doi.org/10.5067/ECOSTRESS/ECO2LSTE.001',
 'Source': 'LP DAAC',
 'TemporalExtentStart': '2018-07-09',
 'TemporalExtentEnd': 'Present',
 'Deleted': False,
 'DOI': '10.5067/ECOSTRESS/ECO2LSTE.001',
 'ProductAndVersion': 'ECO2LSTE.001'}

In [10]:
prodNames = {p['ProductAndVersion'] for p in product_response} # Make list of all products (including version)
for p in prodNames:                                            # Make for loop to search list of products 'Description' for a keyword                
    if 'LST' in products[p]['Product']:
        pprint.pprint(products[p])                             # Print info for each product containing LST in its product description
#pprint is pretty print aesthetically pleasing        


{'Available': True,
 'DOI': '10.5067/ECOSTRESS/ECO2LSTE.001',
 'Deleted': False,
 'Description': 'Land Surface Temperature & Emissivity (LST&E)',
 'DocLink': 'https://doi.org/10.5067/ECOSTRESS/ECO2LSTE.001',
 'Platform': 'ECOSTRESS',
 'Product': 'ECO2LSTE',
 'ProductAndVersion': 'ECO2LSTE.001',
 'RasterType': 'Swath',
 'Resolution': '70m',
 'Source': 'LP DAAC',
 'TemporalExtentEnd': 'Present',
 'TemporalExtentStart': '2018-07-09',
 'TemporalGranularity': 'ISS-dependent',
 'Version': '001'}


In [13]:
prods = ['ECO2LSTE.001']     # Start a list for products to be requested, beginning with ECO2LSTE.001
prods # seems unnecessary to print

['ECO2LSTE.001']

#### I would like explore what layers are within the LST product

In [15]:
lst_response = r.get('{}product/{}'.format(api, prods[0])).json()  # Request layers for the only product (index 0) in the list: ECO2LSTE.001
list(lst_response.keys())

['SDS_Emis1',
 'SDS_Emis1_err',
 'SDS_Emis2',
 'SDS_Emis2_err',
 'SDS_Emis3',
 'SDS_Emis3_err',
 'SDS_Emis4',
 'SDS_Emis4_err',
 'SDS_Emis5',
 'SDS_Emis5_err',
 'SDS_EmisWB',
 'SDS_LST',
 'SDS_LST_err',
 'SDS_PWV',
 'SDS_QC']

#### I'm only interested in SDS_LST right now, so I will use that dictionary key to learn more about that layer from the lst_response variable

In [16]:
lst_response['SDS_LST']

{'AddOffset': 0.0,
 'Available': True,
 'DataType': 'uint16',
 'Description': 'Land Surface Temperature',
 'Dimensions': ['FakeDim0', 'FakeDim1'],
 'FillValue': 0.0,
 'IsQA': False,
 'Layer': 'SDS_LST',
 'OrigDataType': 'uint16',
 'OrigValidMax': 65535,
 'OrigValidMin': 7500,
 'QualityLayers': "['SDS_QC']",
 'QualityProductAndVersion': 'ECO2LSTE.001',
 'ScaleFactor': 0.02,
 'Units': 'K',
 'ValidMax': 1310.7,
 'ValidMin': 150.0,
 'XSize': 5400,
 'YSize': 5632}

In [17]:
# Adding QC layer
layers = [(prods[0],'SDS_LST'),(prods[0],'SDS_QC')]  # Create tupled list linking desired product with desired layers

#### Later I will append more products but I'm keeping this simple for now

#### Here we are created a dictionary list from our tupled list (i.e. layers) and storing each layer+product combination. We will later use this to insert into a json file when we submit our requests.

In [18]:
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })
prodLayer

[{'layer': 'SDS_LST', 'product': 'ECO2LSTE.001'},
 {'layer': 'SDS_QC', 'product': 'ECO2LSTE.001'}]